<a href="https://colab.research.google.com/github/dong-uk-kim97/Exploration/blob/main/Exploration_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요한 라이브러리 불러오기

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import glob, re

txt_file_path = '/content/drive/MyDrive/lyricist/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:20])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough", 'Had some bad love', "So fast love is all that I've got on my mind Ooh, ooh", 'Ooh, ooh Looking for some affirmation', 'Made my way into the sun', 'My friends got their ladies', "And they're all having babies", "I just wanna have some fun I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind']


# 데이터 정제

원하는 문장만 출력

In [2]:
corpus= []

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue           # 길이가 0인 문장 제외
    # if ('[' and ']') in sentence: continue  # 대괄호가 있는 문장을 제외하려고 했으나 안 좋은 결과를 얻었다.
    if sentence[-1] == ":": continue          # 문장 마지막에 ':'로 끝나는 문장 제외
    # if len(sentence.split()) > 20 :continue # maxlen으로 줄여야 했는데 그걸 모르고 이 조건을 사용했더니 안 좋은 결과를 얻었다.
    
    corpus.append(sentence)


print(f'원래 데이터 크기 : {len(raw_corpus)}')
print(f'공백인 문장, 뒤에 \":\" 으로 끝나는 문장 제외한 데이터 크기 : {len(corpus)}')

corpus = list(set(corpus))          # 중복불가한 set의 특성을 이용하여 중복되는 문장 제외 후 다시 리스트로 변환
print(f'중복되는 문장 제외한 데이터 크기 : {len(corpus)}')

원래 데이터 크기 : 187088
공백인 문장, 뒤에 ":" 으로 끝나는 문장 제외한 데이터 크기 : 175749
중복되는 문장 제외한 데이터 크기 : 117960


In [3]:
# 전처리 함수 정의
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                     # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)     # 2
    sentence = re.sub(r'[" "]+', " ", sentence)             # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)   # 4
    sentence = sentence.strip()                             # 5
    sentence = '<start> ' + sentence + ' <end>'             # 6
    return sentence

In [4]:
# 여기에 정제된 데이터를 모은다.
corpus_tokenized = []

for sentence in corpus:
    # 데이터 정제
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus_tokenized.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인
corpus_tokenized[:10]

['<start> don t be afraid to tell me you re sad <end>',
 '<start> u oughta let me come and pet u so it lasts , baby <end>',
 '<start> funny how quick these pricks forget <end>',
 '<start> they ve never seen a fire this hot <end>',
 '<start> i fucked your mom <end>',
 '<start> that from you or me <end>',
 '<start> and wear a shield <end>',
 '<start> word on the street somebody put a hit on me <end>',
 '<start> bitch if you died , wouldn t buy you life <end>',
 '<start> i ve been down <end>']

# 토큰화

In [5]:
# 토큰화
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=14000,         # 14000개 단어를 기억할 수 있음
        filters=' ',
        oov_token="<unk>"        # 포함되지 않는 단어는 <unk> 으로 표현
    )
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=20)  # 토큰 20개 초과 제외
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus_tokenized)

[[   2   35   16 ...    0    0    0]
 [   2   84 4243 ...    0    0    0]
 [   2  767   80 ...    0    0    0]
 ...
 [   2  913    4 ...    0    0    0]
 [   2    7  100 ...    0    0    0]
 [   2   29  203 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f94a9c45110>


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
src_input = tensor[:, :-1] # 소스문장 생성 

tgt_input = tensor[:, 1:]  # 타켓 문장 생성

print(src_input[0])
print(tgt_input[0])

[  2  35  16  27 668  10  91  12   7  56 719   3   0   0   0   0   0   0
   0]
[ 35  16  27 668  10  91  12   7  56 719   3   0   0   0   0   0   0   0
   0]


# 데이터셋 객체 생성 및 전처리

In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 14000개와, 여기 포함되지 않은 0:<pad>를 포함하여 14001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 19), dtype=tf.int32, name=None), TensorSpec(shape=(256, 19), dtype=tf.int32, name=None))>

batch_size는 몇 개의 관측치에 대한 예측을 하고, 레이블 값과 비교를 하는지를 설정하는 파라미터이다. batch_size가 256이면 전체 데이터에 대해 모두 예측한 뒤 실제 레이블 값과 비교한 후 가중치 갱신한다. batch_size가 128이면 128개 데이터에 대해 예측한 뒤 실제 레이블 값과 비교하며 가중치 갱신도 128번 발생한다. \
batch_size가 클수록 많은 데이터를 저장해두어야 하므로 용량이 커진다. 반면, batch_size가 작으면 학습은 촘촘하게 되지만, 계속 레이블과 비교하고, 가중치를 업데이트하는 과정을 거치면서 시간이 오래 걸린다. \
step은 Gradient Descent를 batch_size별로 계산하되, 전체 학습 데이터에 대해서 이를 반복해서 적용한 회수를 말한다. \
 

# 데이터셋 분리

In [10]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2)

# 학습 모델 생성

In [11]:
from tensorflow.keras.layers import Embedding, LSTM, Dense


class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = Embedding(vocab_size, embedding_size)
        self.rnn_1 = LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = LSTM(hidden_size, return_sequences=True)
        self.linear1 = Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear1(out)
        
        return out

# 모델 학습시키기

In [12]:
embedding_size= 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

history = []
epochs = 10

optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

history = model.fit(dataset, 
          epochs=epochs,
          validation_data=(enc_val, dec_val),
          verbose=1)

Epoch 1/10
460/460 [==============================] - 191s 365ms/step - loss: 3.0929 - val_loss: 2.6599
Epoch 2/10
460/460 [==============================] - 167s 363ms/step - loss: 2.5897 - val_loss: 2.4487
Epoch 3/10
460/460 [==============================] - 167s 363ms/step - loss: 2.4337 - val_loss: 2.3043
Epoch 4/10
460/460 [==============================] - 167s 363ms/step - loss: 2.3102 - val_loss: 2.1745
Epoch 5/10
460/460 [==============================] - 167s 362ms/step - loss: 2.1934 - val_loss: 2.0493
Epoch 6/10
460/460 [==============================] - 167s 362ms/step - loss: 2.0753 - val_loss: 1.9251
Epoch 7/10
460/460 [==============================] - 167s 362ms/step - loss: 1.9558 - val_loss: 1.8010
Epoch 8/10
460/460 [==============================] - 167s 363ms/step - loss: 1.8348 - val_loss: 1.6725
Epoch 9/10
460/460 [==============================] - 167s 362ms/step - loss: 1.7124 - val_loss: 1.5522
Epoch 10/10
460/460 [==============================] - 167s 363m

# 문장 생성하고 사람이 평가하기

In [13]:

def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

문장 출력해보기

In [14]:
generate_text(model, tokenizer, init_sentence="<start> anyway", max_len=20)

'<start> anyway , i m gonna be a star <end> '

# 출처

https://ai-rtistic.com/2021/08/20/toy-project-lyricist/#1-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0 \

https://github.com/PEBpung/Aiffel/blob/master/Project/Exploration/E11.%20%EC%9E%91%EC%82%AC%EA%B0%80%20%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%20%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb \

https://sevillabk.github.io/1-batch-epoch/ \

https://bslife.tistory.com/73 



# 회고

데이터셋 객체를 만들고 모델에 적용하지 못해서 많은 시간을 끌었다. steps_per_epoch 값과 실제 step간의 차이를 재은님이 발견하셨고 enc_train, dec_train을 넣었던 자리를 dataset으로 바꾸니 val_loss가 줄어들게 됐다. \
그래서 batch_size, step에 대해 자세히 공부를 해보았고 내가 모델이 내가 원하는 데이터를 학습하고 있는지 확인하기 위해서 step을 확인해봐야겠다는 insight를 얻게 됐다. \
데이터가 잘 학습하고 있는지를 확인한 후에 하이퍼 파라미터(embedding_size, hidden_size)와 내가 작성한 모델을 확인해 봐야겠다는 insight 역시 얻게 됐다.
